In [2]:
import requests
from copy import copy

# Establish baseline
r = requests.get("http://localhost:5000")
ciphertext_baseline = r.cookies["authtoken"]
ciphertext_bytes_baseline = bytearray.fromhex(ciphertext_baseline)

In [3]:
# Determine block size
ciphertext_bytes = copy(ciphertext_bytes_baseline)

def determine_block_size(ciphertext_bytes: bytearray):
    count = 0
    for i in range(len(ciphertext_bytes_baseline)):
        ciphertext_bytes[i] ^= 0x01
        cookie = {"authtoken": ciphertext_bytes.hex()}
        r = requests.get("http://localhost:5000/quote", cookies=cookie)
        
        if (r.content != b'No quote for you!'):
            return count
        
        count += 1

    return -1

block_size = determine_block_size(ciphertext_bytes)
print(f'Block size: {block_size}')

Block size: 16


Note when looking at the ciphertext we use the fact that the last block wont change other blocks since it's the last to find the block size. This means that changes in the cipher here shouldn't result in a change in the rest of the cipher when decoding and therefore it should be able to decode still.

A change in the other parts should change more of the ciphertext resulting in a decode failure.

In [16]:
def oracle(block: bytearray, iv: bytearray, ciphertext_bytes: bytearray):
    block = int.to_bytes(int.from_bytes(block) ^ int.from_bytes(iv), length=block_size)
    start_end = block_count * block_size - block_size * (i_block + 1)
    end_start = block_count * block_size - block_size * i_block

    print(start_end)
    print(end_start)

    ciphertext_bytes = ciphertext_bytes[:start_end] + block + ciphertext_bytes[end_start:]

    cookie = {"authtoken": ciphertext_bytes.hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)

    #print(r.content)
        
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or r.content == b'No quote for you!' or r.content == b'Zero-length input cannot be unpadded'):
        return False
    else:
        return True
        



zero_iv = [0] * block_size

ciphertext_bytes = copy(ciphertext_bytes_baseline)

block_count = len(ciphertext_bytes) // block_size

block = ciphertext_bytes[-16:]
i_block = 0

flag_found = False

for i_block_byte in range(block_size):
    padding_iv = [i_block_byte ^ b for b in zero_iv]
    print(f"Block byte: {i_block_byte}")

    for i in range(256):
        # Guess 1 bytes of the padding iv at a time.
        padding_iv[-(i_block_byte + 1)] = i
        iv = bytes(padding_iv) 

        # Test the padding iv on the block
        if (oracle(block, iv, ciphertext_bytes)):
            if i_block_byte == 0:
                padding_iv[-2] ^= 1
                iv = bytes(padding_iv)

                if not oracle(block, iv, ciphertext_bytes):
                    continue

            flag_found = True
            print(f"Found correct byte at: {i}")
            break

    if (not flag_found):
        pass#raise Exception("no valid padding byte found")
    else:
        flag_found = False
    zero_iv[-(i_block_byte + 1)] = i ^ i_block_byte


Block byte: 0
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112
96
112

KeyboardInterrupt: 

In [90]:
def oracle(block: bytearray, iv: bytearray, ciphertext_bytes: bytearray):
    block = int.to_bytes(int.from_bytes(block) ^ int.from_bytes(iv), length=block_size)

    # Set block at end
    #start_end = block_count * block_size - block_size * (i_block + 1)
    #end_start = block_count * block_size - block_size * i_block
    #ciphertext_bytes = ciphertext_bytes[:start_end] + block + ciphertext_bytes[end_start:]

    # Set block at start
    start_end = block_size * i_block
    end_start = block_size * (i_block + 1)
    ciphertext_bytes = ciphertext_bytes[:start_end] + block + ciphertext_bytes[end_start:]
    #print(len(ciphertext_bytes))
    #print(start_end)
    #print(end_start)
    #print(block)

    cookie = {"authtoken": ciphertext_bytes.hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)

    #print(r.content)
        
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or r.content == b'No quote for you!' or r.content == b'Zero-length input cannot be unpadded'):
        return False
    else:
        return True
        
# Establish baseline
r = requests.get("http://localhost:5000")
ciphertext_baseline = r.cookies["authtoken"]
ciphertext_bytes_baseline = bytearray.fromhex(ciphertext_baseline)

zero_iv = [0] * block_size

ciphertext_bytes = copy(ciphertext_bytes_baseline)

block_count = len(ciphertext_bytes) // block_size

i_block = 0

found_result = False

for i_block in range(block_count):
    block = ciphertext_bytes[len(ciphertext_bytes)-(i_block+1) * block_size:len(ciphertext_bytes)-(i_block) * block_size]
    #print(block.hex())
    for i_block_byte in range(block_size):
        padding_iv = [(i_block_byte + 1) ^ b for b in zero_iv]#[(i_block_byte + 1) ^ b for b in zero_iv]

        for i in range(0, 256):
            padding_iv[-(i_block_byte + 1)] = i
            iv = bytes(padding_iv)
            #print(iv)

            if (oracle(block, iv, ciphertext_bytes)):
                # Check the case where we haven't changed anything yet.
                # The reason for this check is that it will not throw a padding error and think
                # it's correct without doing anything. Therefore we change the byte in front and check if its correct.
                if ((i_block_byte + 1) == 1):
                    padding_iv[-2] ^= 1
                    iv = bytes(padding_iv)

                    if (not oracle(block, iv, ciphertext_bytes)):
                        continue

                found_result = True
                print(i)
                break

            
        if (found_result == False):
            print("Did not find the correct values.")

        print(iv.hex())
        found_result = False

        zero_iv[-(i_block_byte + 1)] = i ^ (i_block_byte + 1)

7c16e897740dc40faa157d0fb1267eb0
0
01010101010101010101010101010000
0
02020202020202020202020202020003
0
03030303030303030303030303000102
0
04040404040404040404040400070605
0
05050505050505050505050001060704
0
06060606060606060606000302050407
0
07070707070707070700010203040506
0
0808080808080808000f0e0d0c0b0a09
0
0909090909090900010e0f0c0d0a0b08
0
0a0a0a0a0a0a0003020d0c0f0e09080b
0
0b0b0b0b0b000102030c0d0e0f08090a
0
0c0c0c0c00070605040b0a09080f0e0d
0
0d0d0d0001060704050a0b08090e0f0c
0
0e0e0003020504070609080b0a0d0c0f
0
0f000102030405060708090a0b0c0d0e
0
001f1e1d1c1b1a191817161514131211
06d3ac786f9dcd99e887e70496719f53
0
110e0f0c0d0a0b080906070405020200
0
120d0c0f0e09080b0a05040706010003
0
130c0d0e0f08090a0b04050607000102
0
140b0a09080f0e0d0c03020100070605
0
150a0b08090e0f0c0d02030001060704
0
1609080b0a0d0c0f0e01000302050407
0
1708090a0b0c0d0e0f00010203040506
0
1807060504030201000f0e0d0c0b0a09
0
1906070405020300010e0f0c0d0a0b08
0
1a05040706010003020d0c0f0e09080b
0
1b04050607000102030c0d